# 5章 Pyの化粧箱：モジュール、パッケージ、プログラム
---
基礎力の養成の段階では、組み込みデータ型からもっと大きなデータ構造、さらにはコード構造の構築に進んだ。この章では、ついに本題に入って、Pythonで現実的な大規模プログラムを書くための方法を学ぶ。

## 5.5
---
Pythonの主張のなかでも目立つもののひとつが「バッテリー同梱」だ。Pythonには、さまざまな役に立つ仕事をしてくれるモジュールを集めた大規模な標準ライブラリがあり、コア言語が膨れ上がるのを防ぐために、別に管理されている。Pythonのコードを書こうとするときには、まず、求めていることをすでに実行している標準モジュールを探してみるとよい。標準ライブラリできらりと光る宝石のようなコードを見つけることは驚くほど多い。

Pythonは、[モジュールの公式ドキュメント集](https://docs.python.jp/3/library/)、[標準ライブラリチュートリアル](https://docs.python.jp/3/tutorial/stdlib.html)も提供している。

本書のこれからの章では、ウェブ、システム、データベース等々の個別分野を対象とする標準モジュールを多数使っていく。この節では、汎用的に使える標準モジュールの一部を取り上げる。

### 5.5.1 setdefault()とdefaultdict()による存在しないキーの処理
---
すでに説明したように、存在しないキーで辞書にアクセスしようとすると例外が生成される。辞書のget()関数を使って、キーが存在しない場合はデフォルト値を返すようにすれば、例外を避けられる。setdefault()関数はget()と似ているが、キーがなければさらに辞書に要素を追加するところが異なる。

In [1]:
periodic_table = {'Hydrogen': 1, 'Helium': 2}
print(periodic_table)

{'Hydrogen': 1, 'Helium': 2}


キーがまだ辞書になければ、新しい値とともに辞書に追加される。

In [2]:
carbon = periodic_table.setdefault('Carbon', 12)
carbon

12

In [3]:
periodic_table

{'Carbon': 12, 'Helium': 2, 'Hydrogen': 1}

既存のキーに別のデフォルト値を代入しようとしても、元の値が返され、辞書は一切変更されない。

In [4]:
helium = periodic_table.setdefault('Helium', 947)
helium

2

In [5]:
periodic_table

{'Carbon': 12, 'Helium': 2, 'Hydrogen': 1}

defaultdict()も例外を防ぐという点では似ているが、辞書作成時にあらゆる新キーのためにあらかじめデフォルト値を設定するところが異なる。引数は関数である。次の例ではint関数を渡している。このintはint()という形で呼び出され、整数の0を返す。

In [6]:
from collections import defaultdict
periodic_table = defaultdict(int)

これで、存在しないキーに対する値は整数（int）の0になる。

In [7]:
periodic_table['Hydrogen'] = 1
periodic_table['Lead']

0

In [8]:
periodic_table

defaultdict(int, {'Hydrogen': 1, 'Lead': 0})

defaultdict()に存在しないキーを用いた場合、そのキーが自動で生成される。そのキーの値は、defaultdict()に渡された引数の型が設定される。次の例では、値が必要になったときにno_idea()が呼び出される。

In [9]:
from collections import defaultdict


def no_idea():
    return 'Huh?'


bestiary = defaultdict(no_idea)
bestiary['A'] = 'Abominable Snowman'
bestiary['B'] = 'Basilisk'
bestiary['A']

'Abominable Snowman'

In [10]:
bestiary['B']

'Basilisk'

In [11]:
bestiary['C']

'Huh?'

int()、list()、dict()、を使えば、これらの型の空の値を返して存在しないキーのデフォルト値にすることができる。int()は0、list()は空リスト（[]）、dict()は空辞書（{}）を返す。デフォルト値引数を省略すると、新しいキーに与えられるデフォルト値はNoneになる。

なお、lambdaを使えば、defaultdict()呼び出しのなかでデフォルト作成関数を定義出来る。

In [13]:
bestiary = defaultdict(lambda: 'Hoh?')
bestiary['E']

'Hoh?'

intは、独自カウンターを作るための手段になり得る。

In [18]:
from collections import defaultdict


food_counter = defaultdict(int)
for food in ['spam', 'spam', 'eggs', 'spam']:
    food_counter[food] += 1

for food, count in food_counter.items():
    print(food, count)


spam 3
eggs 1


上の例で、food_counterがdefaultdictではなく、通常の辞書だったら、辞書の要素のfood_counter[food]は初期化されていないため、これを初めてインクリメントしようとする度にPythonは例外を生成していただろう。次に示すように、余分な作業が必要になっていたはずだ。

In [17]:
dict_counter = {}
for food in ['spam', 'spam', 'eggs', 'spam']:
    if not food in dict_counter:
        dict_counter[food] = 0
    dict_counter[food] += 1

for food, count in dict_counter.items():
    print(food, count)

spam 3
eggs 1


### 5.5.2 Counter()による要素数の計算
---
カウンタが話題になったが、Pythonの標準ライブラリには、上の例で行ったことだけでなく、さらに多くの機能を持った関数が含まれている。

In [1]:
from collections import Counter


breakfast = ['spam', 'spam', 'eggs', 'spam']
breakfast_counter = Counter(breakfast)
breakfast_counter

Counter({'eggs': 1, 'spam': 3})

most_common()関数は、すべての要素を降順で返す。引数として整数を指定すると、最上位から数えてその個数分だけを表示する。

In [2]:
breakfast_counter.most_common()

[('spam', 3), ('eggs', 1)]

In [3]:
breakfast_counter.most_common(1)

[('spam', 3)]

カウンタを結合することもできる。まず、breakfast_counterの内容を確認しておこう。

In [5]:
breakfast_counter

Counter({'eggs': 1, 'spam': 3})

次に、lunchという新しいリストとlunch_counterというカウンタを作る。

In [6]:
lunch = ['eggs', 'eggs', 'bacon']
lunch_counter = Counter(lunch)
lunch_counter

Counter({'bacon': 1, 'eggs': 2})

ふたつのカウンタを結合する第1の方法は、+を使った加算だ。

In [7]:
breakfast_counter + lunch_counter

Counter({'bacon': 1, 'eggs': 3, 'spam': 3})

片方からもう片方を引くには、-を使う。朝食では使われているが昼食では使われていないものは何か。

In [8]:
breakfast_counter - lunch_counter

Counter({'spam': 3})

朝食では食べないが、昼食では食べるものは何だろうか。

In [9]:
lunch_counter - breakfast_counter

Counter({'bacon': 1, 'eggs': 1})

4章の集合と同様に、積集合演算子の&を使えば、共通要素が得られる。

In [10]:
breakfast_counter & lunch_counter

Counter({'eggs': 1})

積集合は共通要素（'eggs'）を選択し、カウンタは小さい方の値になっている。これは正しい。朝食の卵はひとつだけなので、共通の個数は1ということになる。

最後に、和集合演算子の|を使えば、すべての要素を得ることができる。

In [11]:
breakfast_counter | lunch_counter

Counter({'bacon': 1, 'eggs': 2, 'spam': 3})

ここでも、共通要素の'eggs'をどう処理するか問題になる。加算のときと異なり、和集合はカウンタを加算せず、大きい方の値を使う。

### 5.5.3 OrderedDict()によるキー順のソート
---
今までの多くのサンプルコードでは、辞書のキーの順序は予測不能だということを示してきた。a、b、cというキーをその順序で追加しても、keys()はc、a、bと返してくることがある。次に示すのは、1章ですでに使ったサンプルだ。

In [17]:
quotes = {
    'Moe': 'A wise guy, huh?',
    'Larry': 'Ow!',
    'Curly': 'Nyuk nyuk!',
}

for stooge in quotes:
    print(stooge)

Moe
Larry
Curly


OrderedDict()は、キーが追加された順序を覚えていて、イテレータから同じ順序でキーを返す。(key, value)タプルのシーケンスからOrderedDictを作ってみよう。

In [19]:
from collections import OrderedDict

quotes = OrderedDict([
    ('Moe', 'A wise guy, huh?'),
    ('Larry', 'Ow!'),
    ('Curly', 'Nyuk nyuk!'),
])

for stooge in quotes:
    print(stooge)

Moe
Larry
Curly


### 5.5.4 スタック＋キュー＝デック
---
deque(デックと発音する）は、両端キューのことで、スタックとキューの両方の機能を持っている。シーケンスのどちらの端でも要素を追加、削除できるようにしたいときに便利だ。次のサンプルは、単語の両端から中央に向かって文字をひとつずつ処理し、単語が回文（前から読んでも後ろから読んでも同じように読める文）になっているかどうかをチェックする。popleft()はデックから左端の要素を削除して返す。pop()は右端の要素を削除して返す。これらを組み合わせれば、両端から中央に向かって文字をひとつずつ処理できる。両端の文字が等しければ、中央に到達するまで文字の削除を続けていく。

In [1]:
def palindrome(word):
    from collections import deque
    dq = deque(word)
    while len(dq) > 1:
        if dq.popleft() != dq.pop():
            return False
    return True


In [2]:
palindrome('a')

True

In [3]:
palindrome('racecar')

True

In [4]:
palindrome('')

True

In [5]:
palindrome('halibut')

False

このコードはデックの使い方の単純な例として使ったまでであり、高速な回文チェッカーが本当に必要なら、逆順の文字列と比較した方がはるかに簡単だ。Pythonは、文字列で使えるreverse()関数を持っていないが、次のようにスライスを使えば逆順の文字列を作れる。

In [6]:
def another_palindrome(word):
    return word == word[::-1]


In [7]:
another_palindrome('radar')

True

In [8]:
another_palindrome('halibut')

False

### 5.5.5 itertoolsによるコード構造の反復処理
---
[itertools](https://docs.python.org/ja/3/library/itertools.html)には、特別な目的を持つイテレータ関数が含まれている。これらは、for ... inループ内で呼び出されると、一度に1個の要素を返し、呼び出しの間も自分の状態を覚えている。

chain()は、引数全体がひとつのイテラブルであるかのように扱い、そのなかの要素を反復処理する。

In [9]:
import itertools

for item in itertools.chain([1, 2], ['a', 'b']):
    print(item)

1
2
a
b


cycle()は無限反復子で、引数から循環的に要素を返す。

```
import itertools
for item in itertools.cycle([1, 2]):
    print(item)
1
2
1
2
```

これが永遠に続く。止める場合にはCtrl+Cキーを押す。

accumulate()は、要素をひとつにまとめた値を計算する。デフォルトで和を計算する。

In [1]:
import itertools

def multiply(a, b):
    return a * b

for item in itertools.accumulate([1, 2, 3, 4], multiply):
    print(item)

1
2
6
24


itertoolsモジュールは、これら以外にも多くの関数を定義している。特に、順列、組み合わせの関数は、必要な時には時間の節約になる。

### 5.5.6 pprint()によるきれいな表示
---
本書のサンプルは、すべてprint()を使って表示してきた。（あるいは、対話型インタープリタで単に変数名を入力して表示した）。しかし、ときどき結果は見栄えの悪いものになってしまった。pprintのようにきれいに表示してくれるものが必要だ。

In [2]:
from pprint import pprint
from collections import OrderedDict

quotes = OrderedDict([
    ('Moe', 'A wise guy, huh?'),
    ('Larry', 'Ow!'),
    ('Curly', 'Nyuk nyuk!'),
])


ただのprint()は、続けて表示してしまう。

In [3]:
print(quotes)

OrderedDict([('Moe', 'A wise guy, huh?'), ('Larry', 'Ow!'), ('Curly', 'Nyuk nyuk!')])


しかし、pprint()は、読みやすくするために要素の位置を揃えようとする。

In [4]:
pprint(quotes)

OrderedDict([('Moe', 'A wise guy, huh?'),
             ('Larry', 'Ow!'),
             ('Curly', 'Nyuk nyuk!')])
